<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/Deep_Reinforcement_Learning(gym_environment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reference: Nicholas Renotte

#Steps for Deep Reinforcement Learning
1) Creating environments from OpenAi 

2) Building a tf.keras DL model

3) Using keras-rl agents to train

In [ ]:
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install tensorflow==2.3.0

     |████████████████████████████████| 61kB 3.8MB/s 


In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [13]:
states

4

In [14]:
actions

2

In [15]:
episodes = 10
for episode in range (1, episodes+1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    #env.render()
    action = random.choice([0,1])
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score:{}'.format(episode, score))



Episode:1 Score:20.0
Episode:2 Score:14.0
Episode:3 Score:23.0
Episode:4 Score:31.0
Episode:5 Score:12.0
Episode:6 Score:19.0
Episode:7 Score:33.0
Episode:8 Score:24.0
Episode:9 Score:23.0
Episode:10 Score:33.0


In [28]:
import numpy as np
#import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [29]:
def build_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape = (1, states)))
  model.add(Dense(24, activation = 'relu'))
  model.add(Dense(24, activation = 'relu'))
  model.add(Dense(actions, activation = 'linear'))
  return model

In [30]:
model = build_model(states, actions)

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
